https://docs.pinecone.io/docs/quickstart

https://www.sbert.net/

https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens




Loading in CSV data for all publications of all professors in NC State UNC Charlotte, and UNC Chapel Hill

In [ ]:
import pandas as pd

NCSU = pd.read_csv('/content/final_NCSU_data.csv')
UNCC = pd.read_csv('/content/final_UNCC_data.csv')
UNC = pd.read_csv('/content/final_UNC_data.csv')

In [ ]:
NCSU.head()

,Unnamed: 0,Title,Date,Id,Summary,URL,Author
0,0,Generalizing electrocardiogram delineation -- ...,2021-11-25 10:11:41+00:00,http://arxiv.org/abs/2111.12996v2,Obtaining per-beat information is a key task i...,http://arxiv.org/pdf/2111.12996v2,Juan Acosta
1,1,A Generalized Framework for Measuring Pedestri...,2021-05-18 20:22:58+00:00,http://arxiv.org/abs/2105.08814v1,Pedestrian accessibility is an important facto...,http://arxiv.org/pdf/2105.08814v1,Deepti Adlakha
2,2,Using Open Data and Open-Source Software to De...,2022-05-11 02:39:16+00:00,http://arxiv.org/abs/2205.05240v1,Benchmarking and monitoring urban design and t...,http://arxiv.org/pdf/2205.05240v1,Deepti Adlakha
3,3,Oxygen vacancy-induced anomalous Hall effect i...,2021-09-16 16:02:29+00:00,http://arxiv.org/abs/2109.08073v2,"The anomalous Hall effect, a hallmark of broke...",http://arxiv.org/pdf/2109.08073v2,Kaveh Ahadi
4,4,Coexistence of superconductivity and weak anti...,2021-09-20 18:33:17+00:00,http://arxiv.org/abs/2109.09786v1,The intersection of two-dimensional supercondu...,http://arxiv.org/pdf/2109.09786v1,Kaveh Ahadi


Creating a List of all the abstracts and replacing the new line characters

In [ ]:
data = list(NCSU['Summary']) + list(UNCC['Summary']) + list(UNC['Summary'])

data = [y.replace('\n', '') for y in data]

In [ ]:
#Might have to remove latex fomulas and other special characters

data

['Obtaining per-beat information is a key task in the analysis of cardiacelectrocardiograms (ECG), as many downstream diagnosis tasks are dependent onECG-based measurements. Those measurements, however, are costly to produce,especially in recordings that change throughout long periods of time. However,existing annotated databases for ECG delineation are small, being insufficientin size and in the array of pathological conditions they represent. Thisarticle delves has two main contributions. First, a pseudo-synthetic datageneration algorithm was developed, based in probabilistically composing ECGtraces given "pools" of fundamental segments, as cropped from the originaldatabases, and a set of rules for their arrangement into coherent synthetictraces. The generation of conditions is controlled by imposing expert knowledgeon the generated trace, which increases the input variability for training themodel. Second, two novel segmentation-based loss functions have been developed,which attempt

Installing the sentence transformers library to perform embeddings with a pre-trained BERT model

In [ ]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=f1e1d30b6611ce9958a591bd002d42c5862dcb21243796dc69431e08d8fb8815
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

embeddings = model.encode(data)

embeddings

array([[-0.47090787, -0.34158197,  0.38470918, ..., -0.08545293,
        -0.47554964,  0.3649967 ],
       [-0.49331838,  0.4214058 ,  0.5074889 , ...,  0.07310393,
        -0.12202027,  0.06903689],
       [-0.05840614, -0.01529718,  1.0211536 , ..., -0.2641936 ,
         0.16799808,  0.10303508],
       ...,
       [-0.3196099 ,  0.22967564,  0.7998239 , ..., -0.02169785,
        -0.13840786,  0.607152  ],
       [-0.7710979 ,  0.1965076 ,  1.161349  , ..., -0.04780566,
         0.04038496,  0.16807193],
       [-0.19984746,  0.1548395 ,  0.23086819, ..., -0.09329325,
        -0.05783095, -0.40037242]], dtype=float32)

In [ ]:
len(embeddings[1])

768

Setting up a vector database with pinecone API

In [ ]:
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pinecone
pinecone.init(api_key="removed API key for submission", environment="us-west4-gcp-free")

In [ ]:
#pinecone.delete_index("quickstart")
#pinecone.delete_index("nc-univ-abstracts")

pinecone.create_index("quickstart", dimension=768, metric="euclidean")

In [ ]:
index = pinecone.Index("quickstart")

In [ ]:
author_names = list(NCSU['Author']) + list(UNCC['Author']) + list(UNC['Author'])

title = list(NCSU['Title']) + list(UNCC['Title']) + list(UNC['Title'])

Adding all of the embedded abstracts to the database and removing all special characters since the database cannot store those

In [ ]:
import string


ascii_chars = set(string.printable)

for z in range(0, 6001, 100):
  if z == 6000:
    index.upsert([
      (''.join(filter(lambda x: x in ascii_chars, author_names[x] + " " + title[x])), [float(y) for y in list(embeddings[x])]) for x in range(6000,6386)
    ])
  else: 
    index.upsert([
    (''.join(filter(lambda x: x in ascii_chars, author_names[x] + " " + title[x])), [float(y) for y in list(embeddings[x])]) for x in range(z,z+100)
  ])
    



In [ ]:
def get_paragraph_vector(paragraph):
  test_data = word_tokenize(paragraph.lower())
  return [float(x) for x in model.infer_vector(test_data)]

Embedding a test sentence

In [ ]:
sentence = "I am deeply interested in computational chemistry because it offers a powerful toolkit to explore and understand the behavior of molecules and chemical reactions using computer simulations and modeling techniques. The ability to accurately predict molecular structures, energetics, and properties using quantum mechanical calculations fascinates me. I am particularly drawn to the field of drug discovery and design, where computational chemistry plays a crucial role in screening and optimizing potential drug candidates, saving time and resources in the drug development process. The opportunity to contribute to the development of novel computational methods and algorithms that enhance our understanding of chemical phenomena and enable the discovery of new materials and therapeutic agents inspires me to pursue research in computational chemistry."

# Convert the sentence to a vector embedding
embedding = model.encode([sentence])
[float(x) for x in embedding[0]]

[-0.4834110140800476,
 0.169001504778862,
 0.5910281538963318,
 -0.3540003299713135,
 0.2510969340801239,
 -1.081726312637329,
 0.07124583423137665,
 -0.9381856918334961,
 0.8304867744445801,
 -0.6335754990577698,
 0.6101561188697815,
 0.3172786235809326,
 -0.5610018968582153,
 0.5212736129760742,
 -0.10263371467590332,
 0.2027750015258789,
 0.8174994587898254,
 0.013464704155921936,
 -0.14559295773506165,
 -0.25942179560661316,
 0.3581311106681824,
 0.12325866520404816,
 -0.25607332587242126,
 0.9319225549697876,
 0.6840067505836487,
 -0.3887467384338379,
 0.08847934007644653,
 1.1225191354751587,
 0.15239231288433075,
 0.4359714686870575,
 0.024752654135227203,
 -0.9460771083831787,
 -1.0376107692718506,
 0.2995322346687317,
 -0.0977633148431778,
 0.6892662644386292,
 -0.08383104205131531,
 -0.31456804275512695,
 -0.17637695372104645,
 0.2261277288198471,
 -0.4644375443458557,
 -0.36589086055755615,
 0.11404850333929062,
 0.2191469967365265,
 -0.1608125865459442,
 -0.0996646732091903

Getting search results for the test sentence

In [ ]:
[index.query(vector= [float(x) for x in embedding[0]],top_k=10,include_values=True)['matches'][x]['id'] for x in range(10)]

['Paul Zimmerman TorsionNet: A Reinforcement Learning Approach to Sequential Conformer Search',
 'Zixuan Cang A review of mathematical representations of biomolecules',
 'Ehssan Nazockdast A fast platform for simulating flexible fiber suspensions applied to cell mechanics',
 'Lubos Mitas Energetics and Dipole Moment of Transition Metal Monoxides by Quantum Monte Carlo',
 'Junier Oliva MolecularRNN: Generating realistic molecular graphs with optimized properties',
 'ALEXANDER TROPSHA Deep Reinforcement Learning for De-Novo Drug Design',
 'Daniel Munoz SpeedMachines: Anytime Structured Prediction',
 'Justin Baker Quantum Computing at the Frontiers of Biological Sciences',
 'Edward Grant Computation of molecular excited states on IBM quantum computers using a discriminative variational quantum eigensolver',
 'Zixuan Cang Mathematical deep learning for pose and binding affinity prediction and ranking in D3R Grand Challenges']